In [12]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import requests
import time
import json

In [13]:
client = MongoClient("mongodb://localhost:27017/")
db = client.test

In [14]:
def temp_store_memory(agent_id, agents_involved, content):
    memory = {
        "agent_id": agent_id, #!!!
        "agents_involved": agents_involved, #!!!
        "timestamp": datetime.now(), #!!!
        "content": content, #!!!
    }
    db.agent_memories.insert_one(memory)

In [15]:
temp_store_memory("agent_001",
                  ["agent_002"],
                  "I am cooking pasta.")

In [16]:
def temp_retrieve_memories(agent_id):
    memories = db.agent_memories.find({
        "agent_id": agent_id
    }).sort("timestamp", -1)
    return list(memories)

def temp_retrieve_memories_concerning(agent_id):
    memories = db.agent_memories.find({
        "$or": [
            {"agent_id": agent_id},
            {"agents_involved": agent_id}
        ]
    }).sort("timestamp", -1)
    return list(memories)

print(temp_retrieve_memories("agent_001"))

[{'_id': ObjectId('6753412c55a23c2e5a13f5dc'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 6, 19, 23, 40, 126000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('6753410755a23c2e5a13f5da'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 6, 19, 23, 3, 183000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('67533e989f94c4cab0ce42e8'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 6, 19, 12, 40, 296000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('67533e369f94c4cab0ce42e6'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 6, 19, 11, 2, 117000), 'content': 'I am cooking pasta.'}, {'_id': ObjectId('67533ddc9f94c4cab0ce42e4'), 'agent_id': 'agent_001', 'agents_involved': ['agent_002'], 'timestamp': datetime.datetime(2024, 12, 6, 19, 9, 32, 75000), 'content': 'I

In [17]:
class Agent:
    def __init__(self, name, agent_id, user_input, gender):
        self.name = name
        self.agent_id = agent_id 
        self.user_input = user_input
        self.gender = gender
        self.opinions = {}
        
    def __str__(self):
        return f"Agent {self.name} (ID: {self.agent_id}) (Description: {self.user_input}) (Opinions: {self.opinions})"
    
    def name(self):
        return self.name
    
    def agent_id(self):
        return self.agent_id
    
    def user_input(self):
        return self.user_input
    
    def gender(self):
        return self.gender

In [18]:
Sami = Agent("Sami", "agent_001", "Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has an opinion on everything and is kind most of the time.", "male")

Eliana = Agent("Eliana", "agent_002", "Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is the sweetest person on earth and would never lie or attack someone.", "female")

Juliette = Agent("Juliette", "agent_003", "Juliette is a woman who does not know much about cooking, but is willing to learn more about it. She is kind, and pays attention to what others are saying.", "female")

Sami.opinions["Eliana"] = "He likes Eliana and thinks her cooking advices are great"
Sami.opinions["Juliette"] = "He sees Juliette as kind, eager to learn, and attentive, appreciating her willingness to improve her cooking skills"
Eliana.opinions["Sami"] = "She thinks Sami is cool and she likes his cooking"
Eliana.opinions["Juliette"] = "She sees Juliette as kind, open to learning, and a delightful person to share her spice expertise with"
Juliette.opinions["Sami"] = "She admires Sami’s cooking skills and kindness, looking to him for guidance as she learns"
Juliette.opinions["Eliana"] = "She admires Eliana’s knowledge of spices and her sweet, supportive nature, and feels inspired by her wisdom and love for food."

print(Sami)
print(Eliana)
print(Juliette)

Agent Sami (ID: agent_001) (Description: Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has an opinion on everything and is kind most of the time.) (Opinions: {'Eliana': 'He likes Eliana and thinks her cooking advices are great', 'Juliette': 'He sees Juliette as kind, eager to learn, and attentive, appreciating her willingness to improve her cooking skills'})
Agent Eliana (ID: agent_002) (Description: Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is the sweetest person on earth and would never lie or attack someone.) (Opinions: {'Sami': 'She thinks Sami is cool and she likes his cooking', 'Juliette': 'She sees Juliette as kind, open to learning, and a delightful person to share her spice expertise with'})
Agent Juliette (ID: agent_003) (Description: Juliette is a woman who does not know much about cooking, but is willing to learn more about it. She is kind, and pays attention to what others are sa

In [19]:
def make_initial_prompt(agents, subject):
    descriptions = [f"Here is a description of {agent.name}: {agent.user_input}" for agent in agents]
    opinions = [f"Here is what {agent1.name} thinks about {agent2.name}:" + '''
            ''' + f"{agent1.opinions[agent2.name]}" for agent1 in agents for agent2 in agents if agent1 != agent2]
    agent1, agent2 = agents[0], agents[1]
    message_content = f"""
            Context:
            {'''
            '''.join(descriptions)}
            {'''
            '''.join(opinions)}
            The context of this conversation is: {subject}
            Start directly with the conversation, not a introduction phrase. Create the conversation they had:

            {agent1.name}: "
        """
    print(message_content)
    return message_content

def make_subject(memory):
    message_content = f"""
            Context:
            This is the memory of a conversation between 2 people:
            {memory}
            Make a summary in 1 sentence or 2 with the important points and details of the conversation (for example, if they are talking about doing an activity or a dish together, keep it in memory), ignorig the first sentence.
            Start directly with the summary, not with a phrase like "here is a summary".
        """
    return message_content

def extract_emotion(memory, agent_concerned, other):
    message_content = f"""
    Agent1: {agent_concerned.name}, {agent_concerned.gender}, {agent_concerned.user_input}
    Agent2: {other.name}, {other.gender}, {other.user_input}
    Conversation: "{memory}"
    Possible emotions: ["happy", "sad", "angry", "neutral", "fearful"]

    Analyze the information about the agents and their conversation, and identify the primary emotion felt by Agent1 as a result of this interaction.
    The response MUST be a SINGLE word from the provided list of emotions, only from the provided list, nothing more.
    """
    return message_content

def update_opinion(memory, agent_concerned, other):
    message_content = f"""
    {agent_concerned.name}: {agent_concerned.gender}, {agent_concerned.user_input}
    {other.name}: {other.gender}, {other.user_input}
    Conversation: "{memory}"

    Here is the conversation that happened between {agent_concerned.name} and {other.name}.
    Summarize what {agent_concerned.name} thought about {other.name} in one short sentence. The sentence needs to be in third person:
    """
    return message_content

def update_opinion2(opinion, agent_concerned, other, emotion=None):
    message_content = f"""
    {agent_concerned.name}: {agent_concerned.gender}, {agent_concerned.user_input}
    {other.name}: {other.gender}, {other.user_input}
    Last opinion of {agent_concerned.name} about {other.name}: {agent_concerned.opinions[other.name]}
    Analyzed phrase or conversation: "{opinion}"
    {f'Emotion felt by {agent_concerned.name}: "{emotion}"' if emotion else ''}

    Based on the information about both agents and the provided text, update the opinion of {agent_concerned.name} about {other.name}.
    If an emotion is specified, take it into account to adjust the tone or content of this opinion.
    If a part of the old opinion is correct, keep it in the new opinion.
    Juste give the new opinion, nothing else.
    """
    return message_content

In [20]:
url = "http://localhost:1234/v1/chat/completions"
headers = {
    "Content-Type": "application/json"
}

def chat(agents, subject, use_memory = True):
    # First agent's turn
    memories = temp_retrieve_memories(agents[0].agent_id)
    if (len(memories) > 0 and use_memory):
        subject_prompt = make_subject(memories[0])
        data = {
            "model": "llama-3.2-3b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": subject_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }
        response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
        subject = "Here is the resume of the past conversation: " + response + " They meet some time later"
        print("Subject :", subject)
    
    message_prompt = make_initial_prompt(agents, subject)

    data = {
            "model": "llama-3.2-3b-instruct",
            "messages": [
                {"role": "system", "content": ""},
                {"role": "user", "content": message_prompt}
            ],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
            }

    response = requests.post(url, headers=headers, json=data).json()["choices"][0]["message"]["content"]
    print(response)
    #temp_store_memory(agent1.agent_id, [agent2.agent_id], str(response))
    #temp_store_memory(agent2.agent_id, [agent1.agent_id], str(response))
    return response

In [21]:
dialog = chat([Sami, Eliana, Juliette], "They are having an argument about lasagna", False)


            Context:
            Here is a description of Sami: Sami is a man who loves italian food and is learning to cook. He also loves fishing. He has an opinion on everything and is kind most of the time.
            Here is a description of Eliana: Eliana is a woman who loves to eat and sleep. She is very VERY knowledgeable about spices. She is the sweetest person on earth and would never lie or attack someone.
            Here is a description of Juliette: Juliette is a woman who does not know much about cooking, but is willing to learn more about it. She is kind, and pays attention to what others are saying.
            Here is what Sami thinks about Eliana:
            He likes Eliana and thinks her cooking advices are great
            Here is what Sami thinks about Juliette:
            He sees Juliette as kind, eager to learn, and attentive, appreciating her willingness to improve her cooking skills
            Here is what Eliana thinks about Sami:
            She thinks